In [1]:
import pandas as pd
import numpy as np 
import yfinance as yf
import matplotlib.pyplot as plt
import datetime 

In [2]:
#Get the data of the stock
start_date = '2018-03-05'
end_date = '2020-08-16'

boyd = yf.download('BYD', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)

from ta import add_all_ta_features

a = add_all_ta_features(boyd, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
boyd = boyd[["trend_macd", 'momentum_rsi',"Volume" , "Adj Close"]]

footfall = pd.read_csv("casino.csv", names = ['Date', 'Casino1','Casino2','Casino3','Casino4','Casino5','Casino6','Casino7','Casino8','Casino9']) 
footfall = footfall.iloc[5:]

Date = footfall["Date"]
footfall = footfall.drop(["Date"], axis=1)
footfall.index = pd.date_range('20180305', '20200816')
footfall = footfall.astype('int')
footfall["total_foot"] = footfall.sum(axis = 1)
footfall = footfall.drop(['Casino1','Casino2','Casino3','Casino4','Casino5','Casino6','Casino7','Casino8','Casino9'], axis=1)

boyd = boyd.merge(footfall, left_index=True, right_index=True, how='right')

#casino_features = ['Casino1','Casino2','Casino3','Casino4','Casino5','Casino6','Casino7','Casino8','Casino9']
casino_features = ["total_foot"]
tech_features = ["trend_macd", 'momentum_rsi',"Volume"]

boyd.head(10)

/Users/shreyavontela/anaconda3/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/shreyavontela/anaconda3/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,trend_macd,momentum_rsi,Volume,Adj Close,total_foot
2018-03-05,0.000000,100.000000,1003700.0,34.671322,3619
2018-03-06,0.028718,100.000000,1166400.0,35.025013,4385
2018-03-07,0.062856,100.000000,1097800.0,35.172375,4354
2018-03-08,0.081707,83.323807,749100.0,35.083961,4749
2018-03-09,0.070816,51.479538,1789200.0,34.779392,5778
2018-03-10,NaN,NaN,NaN,NaN,5602
2018-03-11,NaN,NaN,NaN,NaN,5070
2018-03-12,-0.018297,22.116384,2354100.0,33.796925,4283
2018-03-13,-0.170868,13.495214,2500100.0,32.775154,4286
2018-03-14,-0.248571,28.022092,1757100.0,33.266396,4563


In [3]:
#Doing holidays adjustment

y = boyd["Adj Close"][7:]
Dates = boyd.index[7:]
n_weeks = 0 
for i in range(0,len(Dates),7):
    week = 7 
    x_date = Dates[i] -  datetime.timedelta(days=week)
    new_df = boyd.loc[x_date : x_date + datetime.timedelta(days=week-3)]
    new_df = new_df.fillna(method='ffill')
    new_df = new_df.fillna(method='bfill')
    if sum(new_df.isna().sum()) != 0:
        print("Nan value not getting filled")
        print(i)
    boyd.update(new_df)
    n_weeks = n_weeks + 1 

In [4]:
boyd["Close"] = boyd["Adj Close"]
boyd = boyd.drop(['Adj Close'], axis=1)
boyd = boyd.interpolate(method='linear', limit_direction='forward', axis=0)
boyd['daily_return'] = boyd['Close'].pct_change().fillna(0)
boyd = boyd.drop(['Close'], axis=1)

## Train test split

In [5]:
val = boyd.values
train_sample = int(len(boyd) * 0.9)
train = val[:train_sample, :]
test = val[train_sample : , :]

print(train.shape, test.shape)
train = pd.DataFrame(train)
test = pd.DataFrame(test)



(806, 5) (90, 5)


## Standardization



In [6]:
#Standardizing the data 
X_train = train.loc[:,:3]
y_train = train.loc[:,4:]
X_test = test.loc[:,:3]
y_test = test.loc[:,4:]



mean = np.mean(X_train)
std = np.std(X_train)
X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

mean_y = np.mean(y_train)
std_y = np.std(y_train)
y_train = (y_train - mean_y)/std_y
y_test = (y_test - mean_y)/std_y



In [7]:
training_set_scaled = np.hstack((np.array(X_train), np.array(y_train)))
test_set_scaled = np.hstack((np.array(X_test), np.array(y_test)))
training_set_scaled

array([[ 0.23644049,  3.89179631, -0.45496818, -0.58742331,  0.00948621],
       [ 0.26444161,  3.89179631, -0.25930684, -0.05143174,  0.32319787],
       [ 0.29772766,  3.89179631, -0.34180449, -0.0731233 ,  0.13887105],
       ...,
       [ 0.36892821,  0.23431187,  2.04108691, -2.89022784,  1.15485657],
       [ 0.40954335,  0.43673215,  2.23285988, -2.92731342,  1.11372895],
       [ 0.45015849,  0.63915242,  2.42463286, -2.88742893,  1.07545255]])

## Creating train data

In [8]:
look_back = 7
X_train = []
y_train = []
for i in range(look_back, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-look_back:i,:-1].reshape(1,look_back,-1))
    y_train.append(training_set_scaled[i,-1])
    
X_train = np.concatenate(X_train, axis = 0 )
y_train = np.asarray(y_train)


    
X_test = []
y_test = []
for i in range(look_back, len(test_set_scaled)):
    X_test.append(test_set_scaled[i-look_back:i,:-1].reshape(1,look_back,-1))
    y_test.append(test_set_scaled[i,-1])
    
X_test = np.concatenate(X_test, axis = 0 )
y_test = np.array(y_test)

In [9]:
X_train.shape

(799, 7, 4)

In [10]:
y_train.shape

(799,)

## Model Building 

The three dimensions of this input are:

Samples. One sequence is one sample. A batch is comprised of one or more samples.
Time Steps. One time step is one point of observation in the sample.
Features. One feature is one observation at a time step.

- (1,7,4)

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM , SimpleRNN, GRU
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow import keras

In [12]:
regressor = Sequential()

regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (7, 4)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 75, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 60))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, verbose = False )

y_pred  = regressor.predict(X_test)


from sklearn.metrics import mean_squared_error
MSE = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test data Root Mean Squared Error is {}'.format(MSE))

MSE = np.sqrt(mean_squared_error(y_train, regressor.predict(X_train)))
print('Train data Root Mean Squared Error is {}'.format(MSE))

Test data Root Mean Squared Error is 1.4358893822602472
Train data Root Mean Squared Error is 0.9504790534123367


In [13]:
regressor = Sequential()

#earlyStopping = EarlyStopping(monitor='mse', patience= 50, verbose=0, mode='min')

regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (7, 4)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 100))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, verbose = False)

y_pred  = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error
MSE = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test data Root Mean Squared Error is {}'.format(MSE))

MSE = np.sqrt(mean_squared_error(y_train, regressor.predict(X_train)))
print('Train data Root Mean Squared Error is {}'.format(MSE))

Test data Root Mean Squared Error is 1.4525604074297878
Train data Root Mean Squared Error is 0.58624507596611


In [19]:
regressor = Sequential()

#earlyStopping = EarlyStopping(monitor='mse', patience= 50, verbose=0, mode='min')

regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (7, 4)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 100))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, verbose = False)

y_pred  = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error
MSE = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test data Root Mean Squared Error is {}'.format(MSE))

MSE = np.sqrt(mean_squared_error(y_train, regressor.predict(X_train)))
print('Train data Root Mean Squared Error is {}'.format(MSE))

Test data Root Mean Squared Error is 1.3549994466375783
Train data Root Mean Squared Error is 0.7505688418009424


In [20]:
regressor = Sequential()

regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (7, 4)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 100))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1))

opt = keras.optimizers.Adam(learning_rate=0.15)
regressor.compile(optimizer = opt, loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, verbose = False)

y_pred  = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error
MSE = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test data Root Mean Squared Error is {}'.format(MSE))

MSE = np.sqrt(mean_squared_error(y_train, regressor.predict(X_train)))
print('Train data Root Mean Squared Error is {}'.format(MSE))

Test data Root Mean Squared Error is 1.4029327586545317
Train data Root Mean Squared Error is 1.1075123738669188


In [21]:
regressor = Sequential()

regressor.add(GRU(100, return_sequences = True, input_shape = (7, 4)))
regressor.add(Dropout(0.3))

regressor.add(GRU(100))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1))

opt = keras.optimizers.Adam(learning_rate=0.18)
regressor.compile(optimizer = opt, loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, verbose = False)

y_pred  = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error
MSE = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test data Root Mean Squared Error is {}'.format(MSE))

MSE = np.sqrt(mean_squared_error(y_train, regressor.predict(X_train)))
print('Train data Root Mean Squared Error is {}'.format(MSE))

Test data Root Mean Squared Error is 1.399650807112742
Train data Root Mean Squared Error is 1.2640447888627213


In [16]:
1.062 * 1.062

1.127844

In [14]:
regressor = Sequential()

regressor.add(GRU(100, return_sequences = True, input_shape = (7, 4)))
regressor.add(Dropout(0.3))

regressor.add(GRU(100))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1))

opt = keras.optimizers.Adam(learning_rate=0.18)
regressor.compile(optimizer = opt, loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 600, batch_size = 64, verbose = False)

y_pred  = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error
MSE = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test data Mean Squared Error is {}'.format(MSE))

MSE = np.sqrt(mean_squared_error(y_train, regressor.predict(X_train)))
print('Train data Root Mean Squared Error is {}'.format(MSE))

Test data Mean Squared Error is 1.0621115781147623
Train data Root Mean Squared Error is 1.0102869508387127


In [20]:
y_pred

array([[0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.11170176],
       [0.111

In [21]:
X_test

array([[[ 0.60852176,  0.87871593,  4.72783265, -2.84194662],
        [ 0.82091447,  1.18308426,  3.24215333, -2.83984743],
        [ 0.97521371,  1.15154881,  1.35697372, -2.84264634],
        ...,
        [ 1.12388912,  1.17558866,  1.15241321, -2.88323056],
        [ 1.16631633,  1.23789524,  1.7843741 , -2.94760553],
        [ 1.20874355,  1.30020182,  2.41633499, -2.91751723]],

       [[ 0.82091447,  1.18308426,  3.24215333, -2.83984743],
        [ 0.97521371,  1.15154881,  1.35697372, -2.84264634],
        [ 1.0814619 ,  1.11328209,  0.52045232, -2.83285015],
        ...,
        [ 1.16631633,  1.23789524,  1.7843741 , -2.94760553],
        [ 1.20874355,  1.30020182,  2.41633499, -2.91751723],
        [ 1.20874355,  1.30020182,  2.41633499, -2.82865179]],

       [[ 0.97521371,  1.15154881,  1.35697372, -2.84264634],
        [ 1.0814619 ,  1.11328209,  0.52045232, -2.83285015],
        [ 1.12388912,  1.17558866,  1.15241321, -2.88323056],
        ...,
        [ 1.20874355,  1.30

In [23]:
from matplotlib import pyplot as plt

boyd = yf.download('BYD', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

pred_dates = y_test.index
open_price = boyd.loc[pred_dates[0]:]["Open"]
y_predicted = y_pred* std_y + mean_y

stock = pd.DataFrame(open_price, index = pred_dates)
stock['Predicted_Close'] = y_predicted
stock['Close'] = y_test * std_y + mean_y
stock["Date"] = stock.index
stock.loc[stock.index[0],'Predicted_Close'] = stock.loc[stock.index[0],'Open'] - 1 

stock['returns_pred']  = stock.Predicted_Close.pct_change()
stock['signal'] = np.where(stock['returns_pred']>0 , 1, 0)
stock['Best_signal'] = np.where(stock['Close']>stock['Open'] , 1, 0)

profit , net = long_only_trading(stock,'signal' ,"Neural Networks Trading Strategy",5000, True, True  )
#profit , net = long_only_trading(stock,'Best_signal' ,"Best possible Trading Strategy",5000, True, True )

#Finding sharpe Ratio 
stock['returns_pred']  = stock.Predicted_Close.pct_change()
stock['excess_daily_ret'] = stock['returns_pred'] - 0.05/252
sharpe = np.sqrt(30) * stock['excess_daily_ret'].mean() / stock['excess_daily_ret'].std()
print("Sharpe Ratio is : "+ str(sharpe) )



AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [29]:
train_sample = int(len(boyd) * 0.9)
train = val[:train_sample, :]
test = val[train_sample : , :]

test.shape

(340, 5)

In [32]:
len(boyd) - len(boyd)*0.9

61.799999999999955

In [34]:
val.shape


(896, 5)

In [35]:
len(boyd)

618

In [22]:
def long_only_trading(stock,signal,title,budget=15000, show_graph=True, show_return=True):
    
    state_check = lambda x, y: 1 if x > y else 0
    stock = stock.reset_index()    
    money = budget
    prev_state = state_check(stock.loc[0].Predicted_Close, stock.loc[0].Open)
    stock_amt = 0
    net = []
    
    for i in range(1, len(stock)):

        today = stock.iloc[i]
        if today[signal] == 1 : 
            #Buy at open price
            stock_amt = int(money / today.Open)
            money = 0
            net.append([today.Date, today.Open, 1, stock_amt * today.Close])
            
            
            #Sell at close price
            money = stock_amt * today.Close
            stock_amt = 0
            net.append([today.Date, today.Close, 0, money])
    
    profit = net[-1][3] - budget
    
    if show_return == True:
        print('Number of Trades: {}'.format(len(net)))
        print('Time Frame: {} days'.format((net[-1][0] - stock.loc[0].Date).days))
        print('Profit: ${:.2f} | {}%'.format(profit, round(profit/budget*100, 2)))

    if show_graph == True:
        trading_history(stock.set_index('Date'), net,title )

    return profit, net    
    
    
    
def trading_history(stock, net,title, std=2):    
    
    # Bollinger band plot with EMA and original historical data
    plt.figure(figsize=(10,5))
    plt.style.use('seaborn-whitegrid')
    plt.plot(stock.index, stock.Close, color='#3388cf', label='Close Price')
    plt.plot(stock.index, stock.Open, color='#ffa33f', label='Open Price')
  
    plt.title(title, fontSize=15)
    plt.ylabel('Price', fontSize=12)
    plt.xlim([stock.index[0], stock.index[-1]])
    
    for i in net:
        if i[2] == 1: 
            color = 'green'
            marker = '^'
        else: 
            color = 'red'
            marker = "v"
        plt.plot_date(i[0], i[1], color=color, marker = marker )
        
    plt.plot_date([],[],label='Buy', c='green', marker = '^')
    plt.plot_date([],[],label='Sell', c='red', marker = "v")
        
    plt.legend(frameon=True, loc=0, ncol=1, fontsize=10, borderpad=.6)
    plt.savefig(title + ".png")
    plt.show()